# <center>Bad Bank Behavior<br>Analyzing Bank Mortgage during the 2007 Housing Bubble</center>  

<center>Michael Siebel</center>
<center>August 2020</center>

<br>
    
## Table of Contents
- [Goals](#Goals)<br>
- [Load Packages](#Load-Packages)<br>
- [Set Up Functions](#Set-Up-Functions)<br>
- [Implement Data Cleanings](#Implement-Data-Cleanings)<br>
- [Analysis Functions](#Analysis-Functions)<br>
- [Imbalanced Prediction](#Imbalanced-Prediction)
- [Downsampling Prediction](#Downsampling-Prediction)<br>
- [Upsampling Prediction](#Upsampling-Prediction)<br>
- [Conclusion](#Conclusion)<br>

# Goals  
<br>

 

***

# Load Packages

In [1]:
# Core Packages
import pandas as pd
import numpy as np
import random
# Convert Time Features
from datetime import datetime as dt
# Data Visualizations
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme='chesterish', grid=False)
# Splitting Data
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# Machine Learning Packages
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# Save Runtime
import time
# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
# Output Statistics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

***

# Set Up Functions  
<br>

In this post, I set up all data wrangling and data analysis as a series of functions, which will enable me to reuse on data from subsequent years (future projects) and various analysis techniques (this project).

### Fannie Mae data comes in two forms:  
### 1) acquistion data and 2) performance data  
<br>
1) The acquisition data includes one observation for each loan with each feature representing knowledge Fannie Mae has when acquiring the loan (e.g., balance, primary lender, credit score, etc.).  

2) The performance data includes observations for each month each loan is held and information on the payment of the loan.  
<br>
I use the acquisition data as predictors for a dichotomous categorization of whether the homeowner defaulted on their loan, a target variable I create using the performance data and merging onto the acquisition data.  

In [2]:
# Load the data
def load_data(acq, per):
    df_acq = pd.read_csv(acq, sep='|', header=None)
    df_per = pd.read_csv(per, sep='|', header=None)
    
    # Specify the name of the columns
    df_acq.columns = ['Loan ID','Origination Channel','Bank','Original Interest Rate',
                      'Original Mortgage Amount','Original Loan Term','Original Date','First Payment',
                      'Original Loan-to-Value (LTV)','Original Combined Loan-to-Value (CLTV)',
                      'Number of Borrowers','Original Debt to Income Ratio','Credit Score',
                      'First Time Home Buyer','Loan Purpose','Property Type','Number of Units',
                      'Occupancy Type','Property State','Zip Code','Mortgage Insurance %',
                      'Product Type','Co-Borrower Credit Score','Mortgage Insurance Type',
                      'Relocation Mortgage Indicator']
    
    df_per.columns = ['Loan ID','MonthRep','Servicer','CurrInterestRate','CAUPB','LoanAge','MonthsToMaturity',
              'AdMonthsToMaturity','MaturityDate','MSA','CLDS','ModFlag','ZeroBalCode','ZeroBalDate',
              'LastInstallDate','Foreclosure Date','DispositionDate','FCCCost','PPRC','AssetRecCost','MHRC',
              'ATFHP','NetSaleProceeds','CreditEnhProceeds','RPMWP','OFP','NIBUPB','PFUPB','RMWPF',
              'FPWA','ServicingIndicator']
    
    return df_acq, df_per

Performance data is much larger as it is transaction based, while the acquistion data has the loan owner as its unit of analysis.  
<br>
I retain only the most recent performance transaction relating to foreclosure, then drop all other variables except  Loan ID (the primary key) and merge performance data onto acquisition data.  
<br>
I recode performance data into dichotomous categorization of whether loan was foreclosed upon.

In [3]:
# Merge foreclosures from performance data to acquistion data
def merge_df(df_acq, df_per):
    
    # Columns to maintain from Performance Data
    per_ColKeep = ['Loan ID','Foreclosure Date']
    df_per = df_per[per_ColKeep]
    df_per = df_per.drop_duplicates(subset='Loan ID', keep='last')
    df = pd.merge(df_acq, df_per, on='Loan ID', how='inner')
    
    # Set Foreclosed to binary
    df.loc[df['Foreclosure Date'].isnull(), 'Foreclosed'] = 0
    df.loc[df['Foreclosure Date'].notnull(),'Foreclosed'] = 1
    df = df.drop(columns='Foreclosure Date')
    df['Foreclosed'] = df['Foreclosed'].astype(int)
    
    # Drop mergeID column
    df = df.drop(df.columns[[0]], axis=1)
    
    return df

I remove features with really high missingness or no data variation, and then impute on features with low missingness

In [4]:
def missing_treat(df):
    # Find features with 10% missing or more
    # condition = ( df.isnull().sum(axis=0)/df.shape[0]*100 ) 
    # df_HighMissing = condition > 10 
    
    # Save features that contain missing data
    # df_HighMissing = df_HighMissing.index[df_HighMissing.values == True]
    
    # remove high missing features
    # df = df.drop(labels=df_HighMissing, axis=1)
    
    # remove Product Type, it only had one value    
    # df = df.drop(labels=['Product Type'], axis=1)
    
    # remove Original Loan-to-Value (LTV) and use Original Combine Loan-to-Value (CLTV)
    df = df.drop(labels=['Original Loan-to-Value (LTV)'], axis=1)
        
    # remove First Payment as this is of no value
    df = df.drop(labels=['First Payment'], axis=1)
        
    # impute on the mean for low missing features that are continuous   
    # df_cont = df.select_dtypes(include=['float64', 'int64'])
    # df[df_cont.columns] = df_cont.apply(lambda x: x.fillna(x.mean()),axis=0)
    
    # impute on the mode for low missing features that are categorical   
    # df_cat = df.select_dtypes(include=['object'])
    # df[df_cat.columns] = df_cat.apply(lambda x: x.fillna(x.mode()),axis=0)  
    
    return df

Changing date features to numeric

In [5]:
# Changing date features to ordinal variables using the function toordinal()
def change_date(df):
    
    # Origination Date
    df['Original Date'] = df['Original Date'].apply(lambda x: dt.strptime(x, '%m/%Y').toordinal())
    
    # Date of First Payment
    # df['FirstPayment'] = df['FirstPayment'].apply(lambda x: dt.strptime(x, '%m/%Y').toordinal())
    
    return df

One Hot Encoding

In [6]:
# Converting categorical variables to dummy variables
def onehotencoding(df):
    columns = df.columns[df.isnull().any()]
    nan_cols = df[columns]

    df = df.drop(nan_cols.columns, axis=1)

    df_cat = df.select_dtypes(include=['object'])
    onehot = pd.get_dummies(df_cat)
    
    df_cont = df.drop(df_cat.columns, axis=1)

    df = pd.concat([df_cont,onehot,nan_cols], axis=1).reset_index(drop=True)
    return df

Function to plot target variable

In [7]:
# Check the percentage and frequency of target variable
def target_values(df_depvar, data=False, prediction=False):
        
    # save target frequencies
    target_frequency = df_depvar.value_counts()
        
    # save target percentage
    target_percentage = round((df_depvar.value_counts()/df_depvar.count())*100).astype(int)
        
    # graphing target variable
    jtplot.style(ticks=True, grid=False)
    plt.figure(figsize=(14,4))
    target_percentage.plot.barh(stacked=True, color='#ca2c92').invert_yaxis()
    if data:
        plt.suptitle('Bar Chart of Target Variable', fontsize=18)
    elif prediction:
        plt.suptitle('Bar Chart of Predictions', fontsize=18)
    else:
        plt.suptitle('Percent of Mortage Defaults', fontsize=18)
    plt.ylabel('Foreclosed')
    plt.xlabel('Percentage')
    plt.xlim([0,100])
    # plt.yticks([0, 1], ['Did not Foreclose', 'Foreclosed'])
    plt.show()
    
    # display frequency of foreclosures
    print('Frequency of Foreclosures\n', target_frequency, '\n', sep='')
    
    # display percentage of foreclosures
    print('Percentage of Foreclosures\n', target_percentage, '\n', sep='')

***

# Implement Data Cleanings

In [ ]:
# Import packages
import os
import glob

# Collect file names
fld = 'C:/Users/siebe/Documents/03_GWU/10 Capstone/Data/'
x = []
for file in os.listdir(fld):
    filename = os.fsdecode(file)
    if filename.startswith("Acquisition_2007Q"): 
        x.append(fld + filename)
        
y = []
for file in os.listdir(fld):
    filename = os.fsdecode(file)
    if filename.startswith("Performance_2007Q"): 
        y.append(fld + filename)

# Load data
df_acq = pd.DataFrame()
df_per = pd.DataFrame()
for i in range(len(x)):
    acq, per = load_data(x[i], y[i])
    df_acq = pd.concat([df_acq, acq],ignore_index=True)
    df_per = pd.concat([df_per, per],ignore_index=True)

                           

In [ ]:
# Shape
df_acq.shape

In [ ]:
# Dates
df_acq['Original Date'].value_counts()

In [ ]:
# Co-Borrower Credit Score
df_acq['Harmonized Credit Score'] = ( df_acq['Co-Borrower Credit Score'][df_acq['Co-Borrower Credit Score'].notnull()] * 0.25 ) \
                                  + ( df_acq['Credit Score'][df_acq['Co-Borrower Credit Score'].notnull()] * 0.75 ) 
df_acq['Harmonized Credit Score'][df_acq['Co-Borrower Credit Score'].isnull()] = df_acq['Credit Score'][df_acq['Co-Borrower Credit Score'].isnull()]    

print(df_acq[['Harmonized Credit Score', 'Credit Score', 'Co-Borrower Credit Score']].head(10))
df_acq = df_acq.drop(labels=['Co-Borrower Credit Score'], axis = 1)

In [ ]:
# Mortgage Insurance %
df_acq['Mortgage Insurance %'] = np.where(df_acq['Mortgage Insurance %'].isnull(), \
                                             0, df_acq['Mortgage Insurance %'])
df_acq['Mortgage Insurance Type'][df_acq['Mortgage Insurance %']==0].value_counts()

In [ ]:
# Mortgage Insurance Type
df_acq['Mortgage Insurance Type'] = np.where(df_acq['Mortgage Insurance Type'].isnull(), \
                                             0, 1)
df_acq['Mortgage Insurance Type'].value_counts()

In [ ]:
# Recode Number of Borrowers
# Single Borrower binary
# More than one borrower is 0
df_acq['Number of Borrowers'] = df_acq['Number of Borrowers'].where(df_acq['Number of Borrowers'] == 1, 0)
df_acq = df_acq.rename(columns={'Number of Borrowers': 'Single Borrower'})
df_acq['Single Borrower'].value_counts()

In [ ]:
# Import zipcode median household income
income = pd.read_excel("C:/Users/siebe/Documents/03_GWU/10 Capstone/Data/est07all.xls",
                       sheet_name = 'est07ALL', header = 2)
income = income[['Name', 'Median Household Income']]
income = income.rename(columns={'Name': 'County'})

# Import county zipcode crosswalk
crosswalk = pd.read_csv("C:/Users/siebe/Documents/03_GWU/10 Capstone/Data/ZIP-COUNTY-FIPS_2017-06.csv",
                        header = 0)
crosswalk = crosswalk[['ZIP', 'COUNTYNAME']]
crosswalk = crosswalk.rename(columns={'ZIP': 'Zip Code', 'COUNTYNAME': 'County'})
crosswalk['Zip Code'] = crosswalk['Zip Code'].astype(str)
crosswalk['Zip Code'] = crosswalk['Zip Code'].str.slice(start=0, stop=-2)
crosswalk['Zip Code'] = crosswalk['Zip Code'].astype(int)

# Merge
income_zipcode = pd.merge(income, crosswalk, on='County', how='outer')
income_zipcode.head(10)

In [ ]:
# Merge zipcode crosswalk with 
income_zipcode = income_zipcode[['Median Household Income', 'Zip Code']]
income_zipcode = income_zipcode.groupby('Zip Code').agg({'Median Household Income': 'mean'})
df_acq = pd.merge(df_acq, income_zipcode, on='Zip Code', how='left')

df_acq['Median Household Income'].describe()

# Merge

In [ ]:
# Create Target Variable with Merge
df = merge_df(df_acq, df_per)

print('\nThe number of features is:\n', df.shape[1], sep='')
print('\nThe number of observations is:\n', df.shape[0], sep='')
target_values(df['Foreclosed'], data=True)

In [ ]:
# Clean Data
## Drop U.S. Terroritories due to missing data
df = df[df['Property State'] != 'PR']
df = df[df['Property State'] != 'GU']
df = df[df['Property State'] != 'VI']
    
## Drop high missing and impute on others
df = missing_treat(df)

pd.options.display.max_columns = None
df.head(10)

In [ ]:
# Missing
(df.isna().sum() / df.shape[0] * 100).round(2)

# Foreclosure Descriptive Statistics

In [ ]:
def Foreclosure_Data(date1 = "", date2 = "", subset = "", df = df):
    # Subset by Date
    if (date1 == "" and date2 == ""):
        df_sub = df
    elif (date1 != "" and date2 != ""):
        date = pd.date_range(date1, date2)
        month_yr = np.array([])
        for i in range(len(date)): 
            month_yr = np.append( month_yr, str( str(date.month[i]) + '/' + str(date.year[i]) ) )
        month_yr = np.unique(month_yr)
        month_yr = np.char.zfill(month_yr, 7)
        df_sub = df[df['Original Date'].isin(month_yr)]
    elif (date2 == ""):
        df_sub = df[df['Original Date']==date1]
    elif (date1 == ""):
        df_sub = df[df['Original Date']==date2]
    
    # Subset by other variable
    if (date1 == "" and date2 == "" and subset != ""):
        df_sub = df[eval(subset)]
        
    # Foreclosures represented
    Foreclosed = ['Not Forclosed', 'Forclosed']
    Target = df_sub.groupby(['Foreclosed']).size().reset_index(name='Total')
    # Original Mortgage Amount
    ORM = df_sub.groupby(['Foreclosed']).agg({'Original Mortgage Amount': 'mean'}).round(2)
    # Credit Score
    CS = df_sub.groupby(['Foreclosed']).agg({'Harmonized Credit Score': 'mean'}).astype(int)
    # Original Debt to Income Ratio
    # DTI = df_sub.groupby(['Foreclosed']).agg({'Original Debt to Income Ratio': 'mean'}).round(1)
    # First Time Home Buyer
    FT = df_sub[df_sub['First Time Home Buyer']=='Y'].groupby(['Foreclosed']).size().reset_index(name='Total')
    # Original Interest Rate
    IR = df_sub.groupby(['Foreclosed']).agg({'Original Interest Rate': 'mean'}).round(2)
    # Original Loan Term
    LT = df_sub.groupby(['Foreclosed']).agg({'Original Loan Term': 'mean'}).astype(int)
    # Original Combined Loan-to-Value (CLTV)
    CLTV = df_sub.groupby(['Foreclosed']).agg({'Original Combined Loan-to-Value (CLTV)': 'mean'}).round(1)
    # Single Borrower Ratio
    SBR = df_sub.groupby(['Foreclosed']).agg({'Single Borrower': 'mean'}).round(2)
    # Mortgage Insurance Type
    MIT = df_sub.groupby(['Foreclosed']).agg({'Mortgage Insurance Type': 'mean'}).round(2)
    # Mortgage Insurance %
    MIP = df_sub.groupby(['Foreclosed']).agg({'Mortgage Insurance %': 'mean'}).round(2)    
    # Median Household Income
    MHI = df_sub.groupby(['Foreclosed']).agg({'Median Household Income': 'mean'}).round(2)
    
    # Create Dataset
    df_new = pd.DataFrame({ 'Foreclosed': Foreclosed, 
                            'Foreclosed (%)': ((Target['Total'] / df_sub.shape[0]) * 100).round(1),
                            'Foreclosed (N)': df_sub.groupby(['Foreclosed']).size(),
                            'Mortgage Amount ($)': ORM['Original Mortgage Amount'].tolist(),
                            'Harmonized Credit Score': CS['Harmonized Credit Score'].tolist(),
                            # 'Debt to Income Ratio': DTI['Original Debt to Income Ratio'].tolist(),
                            'First Time Home Buyer (%)': ((FT['Total'] / Target['Total']) * 100).round(1).tolist(),
                            'Interest Rate': IR['Original Interest Rate'].tolist(),
                            'Loan Term': LT['Original Loan Term'].tolist(),
                            'Combined Loan-to-Value (CLTV)': CLTV['Original Combined Loan-to-Value (CLTV)'].tolist(),
                            'Single Borrower Ratio': SBR['Single Borrower'].tolist(),
                            'Mortgage Insurance Ratio': MIT['Mortgage Insurance Type'].tolist(),
                            'Mortgage Insurance %': MIP['Mortgage Insurance %'].tolist(),
                            'Estimated Household Income ($)': MHI['Median Household Income'].tolist()
                         })
    
    df_new = df_new.set_index('Foreclosed')
    return df_new

Foreclosed = Foreclosure_Data(subset = "df['Property State']=='FL'")
Foreclosed

# Bank Descriptive Statistics

In [ ]:
def Bank_Data(date1 = "", date2 = "", subset ="", df = df):
    # Subset by Date
    if (date1 == "" and date2 == ""):
        df_sub = df
    elif (date1 != "" and date2 != ""):
        date = pd.date_range(date1, date2)
        month_yr = np.array([])
        for i in range(len(date)): 
            month_yr = np.append( month_yr, str( str(date.month[i]) + '/' + str(date.year[i]) ) )
        month_yr = np.unique(month_yr)
        month_yr = np.char.zfill(month_yr, 7)
        df_sub = df[df['Original Date'].isin(month_yr)]
    elif (date2 == ""):
        df_sub = df[df['Original Date']==date1]
    elif (date1 == ""):
        df_sub = df[df['Original Date']==date2]
    
    # Subset by other variable
    if (date1 == "" and date2 == "" and subset != ""):
        df_sub = df[eval(subset)]
        
    # Banks represented
    Banks = df_sub.groupby(['Bank']).size().reset_index(name='Total')
    # Foreclosures
    Target = df_sub.groupby(['Bank']).agg({'Foreclosed': 'mean'})
    # Original Mortgage Amount
    ORM = df_sub.groupby(['Bank']).agg({'Original Mortgage Amount': 'mean'}).round(2)
    # Credit Score
    CS = df_sub.groupby(['Bank']).agg({'Credit Score': 'mean'}).astype(int)
    # Original Debt to Income Ratio
    DTI = df_sub.groupby(['Bank']).agg({'Original Debt to Income Ratio': 'mean'}).round(1)
    # First Time Home Buyer
    FT = df_sub[df_sub['First Time Home Buyer']=='Y'].groupby(['Bank']).size().reset_index(name='Total')
    # Original Interest Rate
    IR = df_sub.groupby(['Bank']).agg({'Original Interest Rate': 'mean'}).round(2)
    # Original Loan Term
    LT = df_sub.groupby(['Bank']).agg({'Original Loan Term': 'mean'}).astype(int)
    # Original Combined Loan-to-Value (CLTV)
    CLTV = df_sub.groupby(['Bank']).agg({'Original Combined Loan-to-Value (CLTV)': 'mean'}).round(1)
    # Single Borrower Ratio
    SBR = df_sub.groupby(['Bank']).agg({'Single Borrower': 'mean'}).round(2)
    # Mortgage Insurance Type
    MIT = df_sub.groupby(['Bank']).agg({'Mortgage Insurance Type': 'mean'}).round(2)
    # Mortgage Insurance %
    MIP = df_sub.groupby(['Bank']).agg({'Mortgage Insurance %': 'mean'}).round(2)     
    # Median Household Income
    MHI = df_sub.groupby(['Bank']).agg({'Median Household Income': 'mean'}).round(2)
    
    # Create Dataset
    df_new = pd.DataFrame({ 'Bank': Banks['Bank'], 
                            'Bank (%)': ((Banks['Total'] / df_sub.shape[0]) * 100).round(1),
                            'Bank (N)': Banks['Total'],
                            'Foreclosed (%)': ((Target['Foreclosed'] * 100).round(1)).tolist(), 
                            'Mortgage Amount ($)': ORM['Original Mortgage Amount'].tolist(),
                            'Credit Score': CS['Credit Score'].tolist(),
                            'Debt to Income Ratio': DTI['Original Debt to Income Ratio'].tolist(),
                            'First Time Home Buyer (%)': ((FT['Total'] / Banks['Total']) * 100).round(1).tolist(),
                            'Interest Rate': IR['Original Interest Rate'].tolist(),
                            'Loan Term': LT['Original Loan Term'].tolist(),
                            'Combined Loan-to-Value (CLTV)': CLTV['Original Combined Loan-to-Value (CLTV)'].tolist(),
                            'Single Borrower Ratio': SBR['Single Borrower'].tolist(),
                            'Mortgage Insurance Ratio': MIT['Mortgage Insurance Type'].tolist(),
                            'Mortgage Insurance %': MIP['Mortgage Insurance %'].tolist(),                           
                            'Median Household Income ($)': MHI['Median Household Income'].tolist()
                        })
    
    df_new = df_new.set_index("Bank")
    return df_new

Banks = Bank_Data()
Banks

In [ ]:
# Banks represented
Banks[['Bank (%)', 'Bank (N)', 'Foreclosed (%)']]

In [ ]:
def search_Banks(col, df = Banks, func = max, subset = True):
    print(col, func.__name__, "value")
    if (subset): cols = col
    else: cols = df.columns
    values = pd.DataFrame(df[cols][df[col] == func(df[col])])
    return values

search_Banks('Foreclosed (%)', func = max)

In [ ]:
month = [1,2,3]
for mnth in range(1, 13):

    yr = '2007'
    mnth = np.char.zfill(str(mnth), 2)
    print(str(mnth) + '/' + str(yr))
    Banks_mnth = Bank_Data(date1 = str(mnth) + '/' + str(yr))
    tbl = search_Banks('Foreclosed (%)', func = max, df = Banks_mnth[Banks_mnth['Bank (N)'] > 100])
    print(display(tbl[['Foreclosed (%)']]))
    print('')


for yr in range(2004, 2007):
    print('Year', yr)
    Banks_yr = Bank_Data(date1 = '01/' + str(yr), date2 = '12/' + str(yr))
    tbl = search_Banks('Foreclosed (%)', func = max, subset = False, df = Banks_yr[Banks_yr['Bank (N)'] > 100])
    print(display(tbl[['Foreclosed (%)', 'Bank (%)', 'Bank (N)']]))
    print('')


# Modeling

Keep:
AMTRUST BANK
BANK OF AMERICA, N.A.
CITIMORTGAGE, INC.
FIRST TENNESSEE BANK NATIONAL ASSOCIATION
FLAGSTAR CAPITAL MARKETS CORPORATION	
GMAC MORTGAGE, LLC
JPMORGAN CHASE BANK, NATIONAL ASSOCIATION
OTHER	
PNC BANK, N.A.
SUNTRUST MORTGAGE INC.

Small:
BISHOPS GATE RESIDENTIAL MORTGAGE TRUST
FDIC, RECEIVER, INDYMAC FEDERAL BANK FSB
FREEDOM MORTGAGE CORP.
GMAC MORTGAGE, LLC (USAA FEDERAL SAVINGS BANK)
HSBC BANK USA, NATIONAL ASSOCIATION
PHH MORTGAGE CORPORATION (USAA FEDERAL SAVINGS BANK)
THIRD FEDERAL SAVINGS AND LOAN
WELLS FARGO BANK, N.A.

Collapse:
CHASE HOME FINANCE	
CHASE HOME FINANCE (CIE 1)	
CHASE HOME FINANCE, LLC

In [ ]:
# Small loan banks
Small_Loan = ['BISHOPS GATE RESIDENTIAL MORTGAGE TRUST', 
              'FREEDOM MORTGAGE CORP.', 'GMAC MORTGAGE, LLC (USAA FEDERAL SAVINGS BANK)', 
              'HSBC BANK USA, NATIONAL ASSOCIATION', 'PHH MORTGAGE CORPORATION (USAA FEDERAL SAVINGS BANK)', 
              'THIRD FEDERAL SAVINGS AND LOAN', 'WELLS FARGO BANK, N.A.']
df = df.replace({'Bank': Small_Loan}, 'SMALL LOAN BANKS')

# Collapse similar banks
Chase = ["CHASE HOME FINANCE (CIE 1)", "CHASE HOME FINANCE, LLC"]
df = df.replace({'Bank': Chase}, 'CHASE HOME FINANCE')

df['Bank'].value_counts()

In [ ]:
Banks = ['AMTRUST BANK', 'BANK OF AMERICA, N.A.', 'CITIMORTGAGE, INC.', 
         'FDIC, RECEIVER, INDYMAC FEDERAL BANK FSB', 
         'FIRST TENNESSEE BANK NATIONAL ASSOCIATION', 'FLAGSTAR CAPITAL MARKETS CORPORATION', 
         'GMAC MORTGAGE, LLC', 'JPMORGAN CHASE BANK, NATIONAL ASSOCIATION', 'OTHER', 
         'PNC BANK, N.A.', 'SUNTRUST MORTGAGE INC.', 'CHASE HOME FINANCE', 'SMALL LOAN BANKS']

# Variables to drop
dropvars = ['Bank', 'Original Date', 'Credit Score', 'Zip Code', 'Mortgage Insurance Type']

# Bank Datasets
Amtrust_X = onehotencoding( df[df['Bank']=='AMTRUST BANK'] \
    .drop(labels=dropvars, axis=1) )
Amtrust_y = Amtrust_X['Foreclosed']
Amtrust_X = Amtrust_X.drop(labels='Foreclosed', axis=1) 

BoA_X = onehotencoding( df[df['Bank']=='BANK OF AMERICA, N.A.'] \
    .drop(labels=dropvars, axis=1) )
BoA_y = BoA_X['Foreclosed']
BoA_X = BoA_X.drop(labels='Foreclosed', axis=1) 

Citi_X = onehotencoding( df[df['Bank']=='CITIMORTGAGE, INC.'] \
    .drop(labels=dropvars, axis=1) )
Citi_y = Citi_X['Foreclosed']
Citi_X = Citi_X.drop(labels='Foreclosed', axis=1) 

IndyMac_X = onehotencoding( df[df['Bank']=='FDIC, RECEIVER, INDYMAC FEDERAL BANK FSB'] \
    .drop(labels=dropvars, axis=1) )
IndyMac_y = IndyMac_X['Foreclosed']
IndyMac_X = IndyMac_X.drop(labels='Foreclosed', axis=1) 

Tenn_X = onehotencoding( df[df['Bank']=='FIRST TENNESSEE BANK NATIONAL ASSOCIATION'] \
    .drop(labels=dropvars, axis=1) )
Tenn_y = Tenn_X['Foreclosed']
Tenn_X = Tenn_X.drop(labels='Foreclosed', axis=1) 

FlagStar_X = onehotencoding( df[df['Bank']=='FLAGSTAR CAPITAL MARKETS CORPORATION'] \
    .drop(labels=dropvars, axis=1) )
FlagStar_y = FlagStar_X['Foreclosed']
FlagStar_X = FlagStar_X.drop(labels='Foreclosed', axis=1) 

GMac_X = onehotencoding( df[df['Bank']=='GMAC MORTGAGE, LLC'] \
    .drop(labels=dropvars, axis=1) )
GMac_y = GMac_X['Foreclosed']
GMac_X = GMac_X.drop(labels='Foreclosed', axis=1) 

JPMorgan_X = onehotencoding( df[df['Bank']=='JPMORGAN CHASE BANK, NATIONAL ASSOCIATION'] \
    .drop(labels=dropvars, axis=1) )
JPMorgan_y = JPMorgan_X['Foreclosed']
JPMorgan_X = JPMorgan_X.drop(labels='Foreclosed', axis=1) 

Misc_X = onehotencoding( df[df['Bank']=='OTHER'] \
    .drop(labels=dropvars, axis=1) )
Misc_y = Misc_X['Foreclosed']
Misc_X = Misc_X.drop(labels='Foreclosed', axis=1) 

PNC_X = onehotencoding( df[df['Bank']=='PNC BANK, N.A.'] \
    .drop(labels=dropvars, axis=1) )
PNC_y = PNC_X['Foreclosed']
PNC_X = PNC_X.drop(labels='Foreclosed', axis=1) 

SunTrust_X = onehotencoding( df[df['Bank']=='SUNTRUST MORTGAGE INC.'] \
    .drop(labels=dropvars, axis=1) )
SunTrust_y = SunTrust_X['Foreclosed']
SunTrust_X = SunTrust_X.drop(labels='Foreclosed', axis=1) 

Chase_X = onehotencoding( df[df['Bank']=='CHASE HOME FINANCE'] \
    .drop(labels=dropvars, axis=1) )
Chase_y = Chase_X['Foreclosed']
Chase_X = Chase_X.drop(labels='Foreclosed', axis=1) 

Small_X = onehotencoding( df[df['Bank']=='SMALL LOAN BANKS'] \
    .drop(labels=dropvars, axis=1) )
Small_y = Small_X['Foreclosed']
Small_X = Small_X.drop(labels='Foreclosed', axis=1) 

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import EasyEnsembleClassifier

X_train, X_test, y_train, y_test = train_test_split(PNC_X, PNC_y,
                                                    stratify = PNC_y, random_state=0)
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_train, y_train) # doctest: +ELLIPSIS

y_pred = eec.predict(X_test)
print(confusion_matrix(y_test, y_pred))

In [ ]:
"""
Grabs the entire Federal Deposit Insurance Corporation (FDIC) Statistics on
Depository Institutions (SDI) data set.

Note that this is a large data set! There are roughly 85 zip files each of
which is between 40 and 84 MB.

"""
import pandas as pd
import requests

base_url = 'https://www7.fdic.gov/sdi/Resource/AllReps/All_Reports_'

# use pandas to construct a list of quarterly dates
present = '20071231'
datetimes = pd.date_range('20070331', end=present, freq='Q')
dates = datetimes.format(formatter=lambda t: t.strftime('%Y%m%d'))

for date in dates:
    print(date)
    # ...construct the url...
    tmp_url = base_url + date + '.zip'

    # ...make the connection and grab the zipped files...
    tmp_buffer = requests.get(tmp_url)

    # ...save them to disk...
    with open('All_Reports_' + date + '.zip', 'wb') as tmp_zip_file:
        tmp_zip_file.write(tmp_buffer.content)

    print('Done with files for ' + date + '!')


In [ ]:
"""
Grabs the entire Federal Deposit Insurance Corporation (FDIC) institutions data
set which catalogues the history of mergers and acquisitions for all FDIC
regulated instutitions and turns it into a Pandas DataFrame and picles the
object for future use.

"""
import zipfile

import pandas as pd
import requests

# download the data
#base_url = 'https://www7.fdic.gov/IDASP/'
filename = 'institutions2/INSTITUTIONS2.CSV'
#tmp_buffer = requests.get(base_url + filename)
#
#with open(filename, 'wb') as tmp_zip_file:
#    tmp_zip_file.write(tmp_buffer.content)
#
## convert to pandas DataFrame
#tmp_buffer = zipfile.ZipFile(filename)
#tmp_file = tmp_buffer.namelist()[1]

used_cols = ['CERT', 'CHANGEC1']
dtypes = {}
tmp_dataframe = pd.read_csv(filename,
                            usecols=used_cols,
                            )
#tmp_dataframe = pd.read_csv(tmp_buffer.open(tmp_file),
#                            usecols=used_cols,
#                            )


In [ ]:
"""
This script imports the subset of the FDIC SDI data used in the analysis,
converts the data to a Pandas data frame and writes the object to disk.

There are on the order of 50 corrupted observations in the various zip files.
Not clear why there are 90 entries in those rows instead of 89

"""
from datetime import datetime
import glob
import zipfile

import pandas as pd

# use pandas to construct a list of quarterly dates
present = '20071231'
datetimes = pd.date_range('20070331', end=present, freq='Q')

# get a list of zip files over which to iterate
zip_files = glob.glob('*.zip')

# only want to return a subset of cols (save on memory usage!)
used_columns = ['cert', 'repdte', 'asset', 'lnlsnet', 'liab', 'dep', 'eqtot',
                'numemp',
                ]
used_dtypes = {'cert': int, 'repdte': datetime, 'asset': float,
               'lnlsnet': float, 'liab': float, 'eqtot': float, 'dep': float,
               'numemp': float}

# create a container for the individual dataframes
dataframes = []

for zip_file in zip_files[0:4]:

    tmp_buffer = zipfile.ZipFile(zip_file)
    
    # want to work with the assets and liabilities file
    tmp_file = tmp_buffer.namelist()[5]
    
    tmp_dataframe = pd.read_csv(tmp_buffer.open(tmp_file),
                                index_col=['cert', 'repdte'],
                                error_bad_lines=False,  # skips the mangled obs
                                usecols=used_columns,
                                #dtype=used_dtypes,
                                parse_dates=True,
                                )
    
    dataframes.append(tmp_dataframe)

    print('Done with ' + zip_file + '!')

# concatenate the quarterly dataframes into a single data frame
combined_dataframe = pd.concat(dataframes)

# convert units from thousands to billions of USD
combined_dataframe[['asset', 'lnlsnet', 'liab', 'dep', 'eqtot']] /= 1e6

# convert units from nummber of people to thousands of people
combined_dataframe['numemp'] /= 1e3

# convert to panel (major_axis: cert, minor_axis: repdte)
combined_panel = combined_dataframe.to_panel()

# pickle the object for later use!
combined_panel.to_pickle('FDIC_SDI_panel_nominal.pkl')


In [ ]:
combined_dataframe.reset_index().head()

In [ ]:
# load the pickled data
FDIC_SDI_panel = combined_dataframe

# compute the by quarter totals for each measure
totals = FDIC_SDI_panel.sum()

# compute the base quarter totals for each measure
base_qtr='2007-03-31'
totals_base_qtr = totals.copy()
totals_base_qtr[:] = totals[base_qtr]
totals_base_qtr.fillna(method='bfill', inplace=True)

def janicki_prescott_norm(item):
    """
    In order to make sure results are comparable across years, I follow 
    Janicki and Prescott (2006) and deflate and re-scale each measure of bank 
    size by dividing by banking sector totals relative to some base quarter. 
    Specifically, let :math:`S_{i,t}^{raw}` denote the raw size of bank :math:`i`
    in year :math:`t` based on one of the six size measures detailed above. The 
    normalized size of bank :math:`i` relative to the base quarter is defined as
    follows:
             
    .. math::
    
        S_{i,t}^{norm} = \frac{S_{i,t}^{raw}}{\sum_{j}S_{j,t}^{raw}}\sum_{j}S_{i,base}^{raw}
    
    where :math:\sum_{j}S_{j,t}^{raw}` is the banking sector total of some size 
    measure in year :math:`t` (i.e., total banking sector assets in year :math:`t`), 
    and :math:`\sum_{j}S_{j,base}^{raw}` is the banking sector total of the same
    size measure in the base quarter.
    
    """
    return (FDIC_SDI_panel[item] / totals[item]) * totals_base_qtr[item]

# apply the Janicki and Prescott (2006) normalized size measure 
for item in FDIC_SDI_panel.items:
    FDIC_SDI_panel[item] = janicki_prescott_norm(item)
    
# pickle the object for later use!
FDIC_SDI_panel.to_pickle('FDIC_SDI_normed_panel.pkl')


In [ ]:
# Income Groups
income_stats = df['Median Household Income'].describe()

df['Income Category'] = np.nan
df['Income Category'][  df['Median Household Income'] <  income_stats['25%']] = 'Lowest Income'
df['Income Category'][( df['Median Household Income'] >= income_stats['25%'] ) & \
                      ( df['Median Household Income'] <  income_stats['50%'] )] = 'Low-Middle Income'
df['Income Category'][( df['Median Household Income'] >= income_stats['50%'] ) & \
                      ( df['Median Household Income'] <  income_stats['75%'] )] = 'High-Middle Income'
df['Income Category'][  df['Median Household Income'] >= income_stats['75%']] = 'Highest Income'

df.head()

In [ ]:
# Income Groups

df['Income Category'] = np.nan
df['Income Category'][  df['Median Household Income'] <  df['Median Household Income'].quantile(0.2) ] = 'Lowest Income'
df['Income Category'][( df['Median Household Income'] >= df['Median Household Income'].quantile(0.2) ) & \
                      ( df['Median Household Income'] <  df['Median Household Income'].quantile(0.4) )] = 'Low-Middle Income'
df['Income Category'][( df['Median Household Income'] >= df['Median Household Income'].quantile(0.4) ) & \
                      ( df['Median Household Income'] <  df['Median Household Income'].quantile(0.6) )] = 'Middle Income'
df['Income Category'][( df['Median Household Income'] >= df['Median Household Income'].quantile(0.6) ) & \
                      ( df['Median Household Income'] <  df['Median Household Income'].quantile(0.8) )] = 'High-Middle Income'
df['Income Category'][  df['Median Household Income'] >= df['Median Household Income'].quantile(0.8) ] = 'Highest Income'

df.head()

In [ ]:
# Bank Variables
df2 = df

NoBL = df.groupby(['Bank', 'Original Date', 'Zip Code']).size().reset_index(name='Number of Bank Loans')
TBL = df.groupby(['Bank', 'Original Date']).size().reset_index(name='Total Bank Loans')
PBL = pd.merge(NoBL, TBL, on=['Bank', 'Original Date'], how="left")
PBL['Proportion of Bank Loans'] = (PBL['Number of Bank Loans'] / PBL['Total Bank Loans']) * 100
df2 = pd.merge(df2, PBL, on=['Bank', 'Original Date', 'Zip Code'], how="left")

df2.head()

In [ ]:
df2['Proportion of Bank Loans'].value_counts()